# A showcase of the different functionalities of the framework

## Workflow

- **Step 1: Gather the Test Inputs**
  - The test session with all the pictures and coordinates
  - The global coordinates of the device
- **Step 2: check for relevant Reference data**
  - use the global coordinates to find all the reference data that is geo-referenced close enough (GPS precision)
- **Step 3: 2D Check**
  - Compare all the test images against all the reference images
  - Find which session has the highest match rate
  - Find which Image has the highest match rate
  - Calculate the transformation between the two images
  - calculate the inverse transformation to give the test data a Reference global position
- **Step 4: 3D Check**
  - Compare the test mesh against relevant point clouds
  - Compare the test mesh against BIM models
  - Perform a CCP for the final alignment
- **Step 5: Choosing final Position**
  - Use the different results from all the methods to come to a best position
  - Send the Position and rotation back to the device

## Session Functionality
A session contains all the data from the folder: the images, meshes and their locations. It also contains the geo-reference position and rotation

### Importing and Selecting Sessions
Sessions can either be directly imported from a path, or be selected from a parent directory based on the distance to the reference point.

In [ ]:
import numpy as np
import session

# you can import all the close enough sessions from a parent directory 
sessionsFolderLocation = "/Volumes/GeomaticsProjects1/Projects/2025-03 Project FWO SB Jelle/7.Data/21-11 Testbuilding Campus/RAW Data"
referencePoint = np.array([0,0,0])
maxDistance = 10
sessions = session.find_close_sessions(sessionsFolderLocation, referencePoint, maxDistance)

In [ ]:
import session
# you can also import a single session from the directory
sessionDirectory = "/Volumes/GeomaticsProjects1/Projects/2025-03 Project FWO SB Jelle/7.Data/21-11 Testbuilding Campus/RAW Data/Hololens/session-2021-11-25 16-09-47"
singleSession = session.Session().from_path(sessionDirectory)
print (singleSession.__dict__)

### Bounding Area
Each session has a bounding area to determine if the reference position is close enough to the data

In [ ]:
boundingBox = singleSession.get_bounding_box()
boundingRadius = singleSession.get_bounding_radius()

print("The axis aligned bounding box (2x3 array) min and max corner points ", boundingBox)
print ("The bounding radius from the reference Center:", boundingRadius)

### Image transforms
The image transform contains the Image file, and it's transform in session space

In [ ]:
# an image transform is stored in 
image1 = singleSession.imageTransforms[0]
print(image1.__dict__)

#### The Image
The image is stored as an openCV color image.

In [ ]:
from matplotlib import pyplot as plt
import cv2

cv2Image = image1.get_cv2_image()

plt.rcParams['figure.dpi'] = 300
plt.imshow(cv2.cvtColor(cv2Image, cv2.COLOR_BGR2RGB))
plt.title('cv2 Color Image')
plt.show()

##### The Image Camera
An image contains it's camera matrix based on the image resolution and vertical fov

In [ ]:
print("The 3x3 intrinsic Camera matrix: \n", image1.get_camera_matrix())
print("The 3x4 camera projection matrix: \n", image1.get_projection_matrix())

### The Geometries

Each session contains some meshes and points clouds, these are all stored in a geometries array

In [ ]:
import positioning3d as pos3D
import session

sessionDirectory = "/Volumes/Data drive/Documents/Doctoraat Local/PythonDataAlignment/src/positioning/images/session-2021-11-25 16-17-19"
singleSession = session.Session().from_path(sessionDirectory)
#print(singleSession.geometries)
pos3D.show_geometries(singleSession.geometries, True)

## Positioning 2D Functionality
All the different methods and functions to calculate the relative position of a Session using 2D data

### Comparing Images
2 images can be compared against each other to find the corresponding matches and quality of the match.

In [ ]:
import session
from matplotlib import pyplot as plt
import cv2

#import 2 images from a session
sessionDirectory = "/Volumes/Data drive/Documents/Doctoraat Local/PythonDataAlignment/src/positioning/images/session-2021-11-25 11-12-17"
singleSession = session.Session().from_path(sessionDirectory)
image1 = singleSession.imageTransforms[0]
image2 = singleSession.imageTransforms[1]

#Find the matches between the 2 images
import imagematch as im

match = im.ImageMatch(image1, image2)
match.find_matches() #find the matches of the image

#plotting
matchImage = match.draw_image_matches() #get an image with the matches displayed
plt.rcParams['figure.dpi'] = 300
plt.imshow(cv2.cvtColor(matchImage, cv2.COLOR_BGR2RGB))
plt.title('cv2 Color Image')
plt.show()

### Comparing Sessions
You can also compare 2 sessions to each other, depending

In [ ]:
session2Directory = "/Volumes/GeomaticsProjects1/Projects/2025-03 Project FWO SB Jelle/7.Data/21-11 Testbuilding Campus/RAW Data/Hololens/session-2021-11-25 16-17-19"
Session2 = session.Session().from_path(session2Directory)

import positioning2d as pos2D

R,t = pos2D.compare_session(singleSession, Session2)

print(R, "\n", t)

### Calculating the Transformation matrix
The transformation between 2 matched images can be determined using the Essential matrix.

In [ ]:
import imagematch as im

match = im.ImageMatch(image1, image2)
match.find_matches()
match.get_essential_matrix()
matchImage = match.draw_image_inliers()
plt.rcParams['figure.dpi'] = 300
plt.imshow(cv2.cvtColor(matchImage, cv2.COLOR_BGR2RGB))
plt.title('cv2 Color Image')
plt.show()

### Triangulating the Camera Position
When the transformation matrix is calculated, the next step is to determine the camera pose.
Because the Essential matrix is correct up to a scale factor, that scale factor needs to be determined first.

#### Minimum distance between known points
The first method requires 2 reference images and 1 test image. Since the location of the reference image is known and the direction of the test image is also determined, The location of the test image can be calculated by finding the minimal distance between the 2 estimated positions.

In [ ]:
import transform

matchScore2, E2, imMatches2 = ci.compare_image(image1, image2)
matchScore3, E3, imMatches3 = ci.compare_image(image1, image3)

newPos, rot1, pos1, pos2, scale = transform.triangulate_session(image2, image3, E2, E3)

print("Estimated position:", newPos)
print("Actual position:", image1.pos)

#### Incremental Matching
Since the location of the reference images is known, we can calculate the feature scaling by matching 2 reference images. By then matching the test image to one of the matched images, the scale is already known.

In [ ]:
import session

#import 2 images from a session
sessionDirectory = "/Volumes/Data drive/Documents/Doctoraat Local/PythonDataAlignment/src/positioning/images/synth"
singleSession = session.Session().from_path(sessionDirectory)
image1 = singleSession.imageTransforms[0]
image2 = singleSession.imageTransforms[2]
#image1.fov *=2
#image2.fov *= 2

In [ ]:
#Find the matches between the 2 images
import imagematch as ci

match = ci.ImageMatch(image1, image2)
match.find_matches()
match.get_essential_matrix()
#projectionMatrices = match.get_projectionMatrices(True)
#correctKeypoints = match.get_keypoints_from_indices()
#print("F: \n", match.fundamentalMatrix)
print("E: \n", match.essentialMatrix)
print("translation \n", match.translation)
print("rotation \n", match.rotationMatrix)
#print("projectionMatrix: \n", projectionMatrices)

In [ ]:
from matplotlib import pyplot as plt
import cv2

matchImage = match.draw_image_inliers()
plt.rcParams['figure.dpi'] = 300
plt.imshow(cv2.cvtColor(matchImage, cv2.COLOR_BGR2RGB))
plt.title('cv2 Color Image')
plt.show()

In [ ]:
import open3d as o3d
import positioning3d as pos3D
import numpy as np
import copy
import quaternion

#oldPoints = match.triangulate()
match.calculate_scaling_factor()
newPoints = match.triangulate(True)

pcd2 = o3d.geometry.PointCloud()
pcd2.points = o3d.utility.Vector3dVector(newPoints)
#pcd2.transform(image1.get_transformation_matrix())

cam1 = pos3D.create_3d_camera(scale=0.2)
cam1_2 = image1.get_camera_geometry(scale=1)
cam2_2 = image2.get_camera_geometry(scale=1)
cvTranslation = np.array([image1.pos]).T + np.array( - match.rotationMatrix.T @ image1.get_rotation_matrix() @ match.translation)
print("pos:", cvTranslation)
cvRotation = match.rotationMatrix.T @ image1.get_rotation_matrix()
cam2 = pos3D.create_3d_camera(cvTranslation,cvRotation , 0.2)
cam2_Local = pos3D.create_3d_camera(- match.rotationMatrix.T @ match.translation,match.rotationMatrix.T , 1)

cam2_Local_Real = pos3D.create_3d_camera( 
     image1.get_rotation_matrix().T @ (image2.pos - image1.pos), 
     image1.get_rotation_matrix().T @ image2.get_rotation_matrix() , 1)

mesh = singleSession.geometries[0]
visMesh = copy.deepcopy(mesh)
#visMesh.vertices = o3d.utility.Vector3dVector(
#    np.asarray(visMesh.vertices) * np.array([-1., -1., 1.]) )
visMesh.compute_vertex_normals()

pos3D.show_geometries([cam1_2,cam2_2,visMesh,pcd2], False)


In [ ]:
# mget keypoints and descriptors out of the new image
# match the new image with the other image and retrieve the matches
# find which matches are already 
print(match.mask.shape)

In [ ]:
#Adding the 3rd Image
image3 = singleSession.imageTransforms[1]
image3.fov *=2

match3rd = ci.ImageMatch(image2, image3)
#match3rd.find_matches()

print("matches:",np.array(match.matches).shape)
print("positive mask:", np.sum(match.mask))
print("3d points:",match.points3d.shape)

R, t = match3rd.get_pnp_pose(match)

print("R:\n", R, "\nt:\n", -t.T)
print("actual translation:", image3.pos)


#### Using the 3D scene
The session data might also contain 3D data, like a mesh or point cloud. They are also localized, so once the features are calculated for an image, a ray can be cast from a point to determine the global scale.

In [ ]:
import positioning3d as pos3d
import session
import imagematch as ci

#import 2 images from a session
sessionDirectory = "/Volumes/Data drive/Documents/Doctoraat Local/PythonDataAlignment/src/positioning/images/synth"
singleSession = session.Session().from_path(sessionDirectory)
image1 = singleSession.imageTransforms[0]
image2 = singleSession.imageTransforms[1]
#image1.fov *=2
#image2.fov *= 2

useTestPose = True

match = ci.ImageMatch(image1, image2)
match.find_matches() # compute the matches
match.calculate_transformation_matrix() # Calculate the essential matrix
match.triangulate(useTestPose) # determine the 3D points

In [ ]:
import numpy as np
import math

#cast a number of rays on the determined points in the scene
scalingFactors = []

for i, point in enumerate(match.points3d):
    pointVector = point - (image1.pos if useTestPose else 0)
    pointDistance = np.linalg.norm(pointVector)
    direction = pointVector / pointDistance
    rayDistance = pos3d.cast_ray_in_mesh(singleSession.geometries[0], image1.pos, direction)
    if(not math.isinf(rayDistance)):
        scalingFactors.append(rayDistance/pointDistance)

scalingFactor = sum(scalingFactors) / float(len(scalingFactors))

print("Estimated scaling factor:", scalingFactor)

print("Actual scaling factor:", match.calculate_scaling_factor())
# use the ransac om de outliers te filteren 
# mediaan en standardafwijking, om de inliers te bepalen, dan gemiddelde berekenen

    

#### RayCasting against voxelised point clouds
point clouds have no surface, so castign a ray has no meaning. We can however cast a r ray against the surface of the occupied voxels of the pcd

In [ ]:
   
import open3d as o3d
import math

def voxel_traversal(pcd, origin, direction, voxelSize): 
    """Traversal algorithm"""

    #range parameters
    t = 0
    maxRange = 100

    #the starting index of the voxels
    voxel_origin = pcd.get_voxel(origin)
    iX = math.floor(origin[0] * voxelSize) / voxelSize
    iY = math.floor(origin[1] * voxelSize) / voxelSize
    iZ = math.floor(origin[2] * voxelSize) / voxelSize

    stepX = np.sign(direction[0])
    stepY = np.sign(direction[1])
    stepZ = np.sign(direction[2])

    tDeltaX = 1/direction[0] * voxelSize
    tDeltaY = 1/direction[1] * voxelSize
    tDeltaZ = 1/direction[2] * voxelSize

    tMaxX = origin[0]
    tMaxY = origin[1]
    tMaxZ = origin[2]

    for i in range(0,maxRange):
        # check if the current point is in a occupied voxel
        if(pcd.check_if_included(o3d.utility.Vector3dVector([[iX * voxelSize, iY * voxelSize, iY * voxelSize]]))[0]):
            distance = np.linalg.norm(np.array([iX * voxelSize, iY * voxelSize, iY * voxelSize]) - origin)
            return True, distance

        if(tMaxX < tMaxY):
            if(tMaxX < tMaxZ):
                #traverse in the X direction
                tMaxX += tDeltaX
                iX += stepX
            else:
                #traverse in the Z direction
                tMaxZ += tDeltaZ
                iZ += stepZ
        else:
            if(tMaxY < tMaxZ):
                #traverse in the Y direction
                tMaxY += tDeltaY
                iY += stepY
            else:
                #traverse in the Z direction
                tMaxZ += tDeltaZ
                iZ += stepZ

    return False, math.inf
        


In [ ]:
import positioning3d as pos3d
import open3d as o3d
import numpy as np

pcd = pos3d.import_point_cloud("/Volumes/Data drive/Documents/Doctoraat Local/PythonDataAlignment/src/positioning/images/ref/pcd1.pcd")

voxelSize = 1
maxRange = 1000
origin = np.array([0,0,0])
direction = np.array([0,0,0])

pcd_Voxel = o3d.geometry.VoxelGrid.create_from_point_cloud(pcd,voxel_size=voxelSize)
print(pcd_Voxel.get_voxels()[0].grid_index, pcd_Voxel.get_voxel_center_coordinate(pcd_Voxel.get_voxels()[0].grid_index))
print("coordinates",np.asarray(pcd_Voxel.get_voxel_center_coordinate([10,10,10])))
print("query",pcd_Voxel.get_voxel([ 1, 0,  0]))
print("included:", pcd_Voxel.check_if_included(o3d.utility.Vector3dVector([[0,0,0]]))[0])


#dist = pos3d.cast_ray_in_mesh(pcd_Voxel, np.array([0,0,0]), np.array([0,1,1]))

#pos3d.show_geometries([pcd_Voxel])

#Gameplan:
# Get voxel grid, check every voxel from camera in direction and check if in girid, get the position of the hit voxel and set that as distance

## Positioning 3D Functionality
Using mesh data to calculate the transformations, the pipeline uses open3D as the main framework for importing, displaying and matching geometry

### Importing meshes and point clouds


In [ ]:
import positioning3d as pos3D
import open3d as o3d

meshPath = "/Volumes/Data drive/Documents/Doctoraat Local/PythonDataAlignment/src/positioning/images/ref/mesh-2021-11-25 16-16-01.obj"
mesh = pos3D.import_mesh(meshPath)
pos3D.show_geometries([mesh])

In [ ]:
import positioning3d as pos3D

pcdPath = "/Volumes/Data drive/Documents/Doctoraat Local/PythonDataAlignment/src/positioning/images/ref/S003-SW-002.ply"
pcd = pos3D.import_point_cloud(pcdPath)

pos3D.show_geometries([pcd])

#### Converting and Downsampling
Meshes have to be converted to point clouds to extract features

In [ ]:
meshPath = "/Volumes/Data drive/Documents/Doctoraat Local/PythonDataAlignment/positioning/images/ref/mesh-2021-11-25 16-16-01.obj"
mesh = pos3D.import_mesh(meshPath)
nrOfPoints = 10000
superSampleFactor = 2
pcd = pos3D.to_pcd(mesh, nrOfPoints, superSampleFactor)

### Aligning Point clouds
2 point clouds can be aligned using a FPFH 3D feature matching algorithm.

In [ ]:
import positioning3d as pos3D
import copy

pcd1 = pos3D.import_point_cloud("/Volumes/Data drive/Documents/Doctoraat Local/PythonDataAlignment/src/positioning/images/ref/pcd1.pcd")
#pcd2 = pos3D.import_point_cloud("/Volumes/Data drive/Documents/Doctoraat Local/PythonDataAlignment/src/positioning/images/ref/pcd2.pcd")
pcd2 = pos3D.import_point_cloud("/Volumes/Data drive/Documents/Doctoraat Local/PythonDataAlignment/src/positioning/images/ref/S003-SW-002.ply")

transformation, error = pos3D.get_pcd_transformation(pcd1, pcd2, 0.1)
print("This is the estimated transformation: \n", transformation)
movedPcd = copy.deepcopy(pcd1)
movedPcd.transform(transformation)
pos3D.show_geometries([pcd1,movedPcd, pcd2])

## Global Alignment
Once a number of estimated transformations are determined, you can compare them to find the best one

In [ ]:
import numpy as np
import session
import positioning2d as pos2d
import positioning3d as pos3d

# you can import all the close enough sessions from a parent directory 
sessionsFolderLocation = "/Volumes/GeomaticsProjects1/Projects/2025-03 Project FWO SB Jelle/7.Data/21-11 Testbuilding Campus/RAW Data"
referencePoint = np.array([0,0,0])
maxDistance = 10
sessions = session.find_close_sessions(sessionsFolderLocation, referencePoint, maxDistance)

testSession = session.Session().from_path("/Volumes/Data drive/Documents/Doctoraat Local/PythonDataAlignment/src/positioning/images/session-2022-01-14 09-23-42")

poses2d = pos2d.get_2D_transformation(testSession, sessions)
poses3d = pos3d.get_3D_transformation(testSession, sessions)

R,t = testSession.get_best_pose()

print("Global position: \n", t, "\n Glopal rotation: \n", R)

## Full example
This a a workflow example going over all the steps

In [ ]:
import numpy as np
import session

referencePoint = np.array([0,0,0])
maxDistance = 10

testSessionDir = "/Volumes/Data drive/Documents/Doctoraat Local/PythonDataAlignment/src/positioning/images/session-2021-11-25 16-17-19"
refSessionDir = "/Volumes/Data drive/Documents/Doctoraat Local/PythonDataAlignment/src/positioning/images/session-2021-11-25 16-09-47"
testSession = session.Session().from_path(testSessionDir)
referenceSession = session.Session().from_path(refSessionDir)
closeEnoughSessions = [referenceSession]

In [ ]:
#2D estimation
import positioning2d as pos2D
import transform

ImageTransformations = pos2D.get_2D_transformation(testSession, closeEnoughSessions)

for transformation in ImageTransformations:
    print("These are the estimated transformations:")
    E2 = transformation[0].transMatrix
    E3 = transformation[1].transMatrix
    image2 = transformation[0].refImage
    image3 = transformation[1].refImage
    newPos, rot1, pos1, pos2, scale = transform.triangulate_session(image2, image3, E2, E3)

    print("Estimated position:", newPos)
    print("Actual position:", transformation[0].testImage.pos)

In [ ]:
#3D estimation
import positioning3D as pos3D

MeshTransformation = pos3D.get_3D_transformation(testSession, closeEnoughSessions, 0.1)

In [ ]:

import copy

for transformations in MeshTransformation:
    print("here are the estimated transformations:")
    for i, element in enumerate(transformations):
        print(element)
        transMesh = copy.deepcopy(testSession.geometries[0])
        transMesh.transform(element)
        pos3D.show_geometries([transMesh, closeEnoughSessions[0].geometries[i]])

In [ ]:
transMesh = copy.deepcopy(testSession.geometries[0]).transform(element)
pos3D.show_geometries([transMesh, closeEnoughSessions[0].geometries[0]])

In [ ]:
import positioning3D as pos3D
import numpy as np
import copy

meshPath = "/Volumes/Data drive/Documents/Doctoraat Local/PythonDataAlignment/src/positioning/images/session-2021-11-25 16-17-19/mesh-2021-11-25 16-17-19.obj"
mesh = pos3D.import_mesh(meshPath)
nrOfPoints = 100000
superSampleFactor = 1
pcdTest = pos3D.to_pcd(mesh, nrOfPoints, superSampleFactor)

meshPath = "/Volumes/Data drive/Documents/Doctoraat Local/PythonDataAlignment/src/positioning/images/session-2021-11-25 16-09-47/mesh-2021-11-25 16-16-01.obj"
mesh = pos3D.import_mesh(meshPath)
nrOfPoints = 100000
superSampleFactor = 1
pcdRef = pos3D.to_pcd(mesh, nrOfPoints, superSampleFactor)

#pcdRef = pos3D.import_point_cloud("/Volumes/Data drive/Documents/Doctoraat Local/PythonDataAlignment/src/positioning/images/ref/S003-SW-002.ply")

transformation = pos3D.get_pcd_transformation(pcdTest, pcdRef, 0.1)
print("This is the estimated transformation: \n", transformation)
movedPcd = copy.deepcopy(pcdTest)
movedPcd.transform(transformation)
pos3D.show_geometries([pcdTest,movedPcd, pcdRef],True)

## Kerk Example

In [ ]:
import open3d
import cv2
import session
from matplotlib import pyplot as plt
import numpy as np
import compareImage as ci
from transform import ImageTransform
import positioning3D as pos3D
import transform
import open3d as o3d

     

cam1_rot = [-0.4816, -0.7658, 0.4259, -0.7635, 0.1282, -0.6329, 0.43, -0.6300,-0.6465]
cam1_pos = [3.3354, 8.6431, 2.1599]
cam2_rot = [-0.4824, -0.7647, 0.4271, -0.762, 0.1271, -0.6339, 0.4304, -0.6317, -0.6447]
cam2_pos = [4.2553, 9.7217, 5.3615]

image1 = ImageTransform()
image1.path = "/Volumes/Data drive/Documents/Doctoraat Local/PythonDataAlignment/src/positioning/images/kerk/DJI_0017.JPG"
image1.set_transformation_matrix(cam1_pos, cam1_rot)
image1.fov = 105

image2 = ImageTransform()
image2.path = "/Volumes/Data drive/Documents/Doctoraat Local/PythonDataAlignment/src/positioning/images/kerk/DJI_0018.JPG"
image2.set_transformation_matrix(cam2_pos, cam2_rot)
image2.fov = 105

In [ ]:
match = ci.ImageMatch(image1, image2)
match.find_matches()
match.calculate_transformation_matrix()
projectionMatrices = match.get_projectionMatrices()
#correctKeypoints = match.get_keypoints_from_indices()
#print("F: \n", match.fundamentalMatrix)
print("E: \n", match.essentialMatrix)
#print("projectionMatrix: \n", projectionMatrices)

In [ ]:
matchImage = match.draw_image_inliers()
plt.rcParams['figure.dpi'] = 300
plt.imshow(cv2.cvtColor(matchImage, cv2.COLOR_BGR2RGB))
plt.title('cv2 Color Image')
plt.show()

In [ ]:
testPoints = match.testInliers.T
refPoints = match.refInliers.T
#print(testPoints)

points3d = cv2.triangulatePoints(projectionMatrices[0], projectionMatrices[1], testPoints, refPoints)
points3d /= points3d[3]
finalPoints = np.array(points3d[:3]).T
#print(finalPoints)

#ret, rvecs, tvecs, more = cv2.solvePnPRansac(points3d[:3].T, refPoints.T,image2.get_camera_matrix(),np.zeros((1,4)))

cam1 = pos3D.create_3d_camera(scale=0.2)
R = match.rotationMatrix
t = match.translation
cvTranslation = np.array(- R.T @ t)
cvRotation = R.T
print("R: \n", cvRotation,"\n t: \n", cvTranslation)
cam2 = pos3D.create_3d_camera(cvTranslation,cvRotation , 0.2)
#cam2 = pos3D.create_3d_camera(np.array([0,-1,0]), R1, 0.2)
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(finalPoints)
#pos3D.show_geometries([image1.get_camera_geometry(), image2.get_camera_geometry(), pcd])
pos3D.show_geometries([pcd, cam1, cam2])

In [ ]:
#scaling the scene

realTranslation = image2.pos - image1.pos
realDirection = realTranslation / np.linalg.norm(realTranslation)

realLocalTranslation = image1.get_rotation_matrix().T @ realTranslation
realLocalDirection = realLocalTranslation / np.linalg.norm(realLocalTranslation)
print("Rotationmatrix: \n", image1.get_rotation_matrix())
print("local z axis = \n", image1.get_rotation_matrix() @ np.array([0,0,1]))
print("Real translation(lambert): \n", realTranslation, "\n with direction:", realDirection)
print("Real local translation(lambert): \n", realLocalTranslation, "\n with direction:", realLocalDirection)
print("CV translation(Y down): \n", cvTranslation.T, "\n with direction:", cvTranslation.T / np.linalg.norm(cvTranslation.T))


In [ ]:
# adding the 3rd image

cam3_rot = [-0.4729, -0.7733, 0.4222, -0.8479, 0.2692, -0.4566, 0.2394, -0.5739, -0.7830]
cam3_pos = [4.7772, 8.8708, 13.0912]

image3 = ImageTransform()
image3.path = "/Volumes/Data drive/Documents/Doctoraat Local/PythonDataAlignment/src/positioning/images/kerk/DJI_0024.JPG"
image3.set_transformation_matrix(cam3_pos, cam3_rot)
image3.fov = 105

## Room Example

In [ ]:
import session

# refernence bim session
refSession = session.Session().from_path("/Volumes/Data drive/Documents/Doctoraat Local/PythonDataAlignment/src/positioning/images/session-2022-01-20 11-42-58")
# test session
testSession = session.Session().from_path("/Volumes/Data drive/Documents/Doctoraat Local/PythonDataAlignment/src/positioning/images/session-2022-01-14 09-23-42")

In [ ]:
import positioning3d as pos3d
import copy
#3D test

nrOfPoints = 100000
superSampleFactor = 2
pcd1 = pos3d.to_pcd(refSession.geometries[1], nrOfPoints, superSampleFactor)
pcd2 = pos3d.to_pcd(testSession.geometries[0], nrOfPoints, superSampleFactor)
pos3d.show_geometries([pcd1, pcd2], True)


In [ ]:
import positioning3d as pos3d
pcd1 = pos3d.import_point_cloud("/Volumes/Data drive/Documents/Doctoraat Local/PythonDataAlignment/src/positioning/images/ref/pcd1.pcd")
#pcd2 = pos3d.import_point_cloud("/Volumes/Data drive/Documents/Doctoraat Local/PythonDataAlignment/src/positioning/images/ref/pcd2.pcd")
pcd2 = pos3d.import_point_cloud("/Volumes/Data drive/Documents/Doctoraat Local/PythonDataAlignment/src/positioning/images/ref/S003-SW-002.ply")

In [ ]:
import copy
transformation, error = pos3d.get_pcd_transformation(pcd1, pcd2, 0.1)
print("This is the estimated transformation: \n", transformation)
movedPcd = copy.deepcopy(pcd1)
#ransacPcd = copy.deepcopy(pcd1)
#movedPcd.transform(transformation)
#transformation, error = pos3d.get_pcd_transformation(movedPcd.transform(transformation), pcd2, 0.01)
pos3d.show_geometries([movedPcd.transform(transformation), pcd2], True) #,ransacPcd.transform(ransac)

In [ ]:
import positioning3d as pos3d
import open3d as o3d

pcdPath = "/Volumes/Data drive/Documents/Doctoraat Local/PythonDataAlignment/src/positioning/images/ref/S003-SW-002.ply"
pcd = pos3D.import_point_cloud(pcdPath)

#pos3D.show_geometries([pcd])

voxelSize = 0.1
voxel_pcd1 = pcd.voxel_down_sample(voxelSize)
voxel_pcd1.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=voxelSize*2, max_nn=30))
fpfh_pcd1 = pos3d.get_fpfh_features(voxel_pcd1, voxelSize * 5)

In [ ]:
print(fpfh_pcd1.data.T[0])
feature = fpfh_pcd1.data.T[0]